In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor 

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv', parse_dates=['date'])

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)
df.nunique()

In [ ]:
df.drop(['integration_type_id' , 'revenue_share_percent', 'total_revenue'], axis = 1, inplace=True)

удалим отрицательные значения из df

In [ ]:
df = df[df['CPM'] >= 0]

сплит по дате

In [ ]:
split_date = '2019-06-22'

df_train = df[df['date'] < split_date]
df_test = df[df['date'] >= split_date]


удалим топ 5% из трейн и тест

In [ ]:
train_cpm_cut = df_train['CPM'].quantile(0.95)
test_cpm_cut = df_test['CPM'].quantile(0.95)
df_train = df_train[df_train['CPM'] < train_cpm_cut]
df_test = df_test[df_test['CPM'] < test_cpm_cut]
df_train['CPM'].describe()

In [ ]:
y_train, X_train  = df_train['CPM'], df_train.drop(columns = ['CPM']) 
y_test, X_test  = df_test['CPM'], df_test.drop(columns = ['CPM'])

In [ ]:
df_train.nunique()

Закоментированные колонки снизу символизируют ручной подбор, чтобы сделать скор чуть ниже

In [ ]:
cat_col = [
'site_id'
#,'ad_type_id'
#,'geo_id'
,'device_category_id'
#,'advertiser_id'
,'os_id'
#,'monetization_channel_id'
,    'order_id' 
#, 'line_item_type_id'
,'ad_unit_id'
]

num_col = [
#'total_impressions'
#'viewable_impressions'
'measurable_impressions'
]

In [ ]:
num_transform = preprocessing.StandardScaler()

scaled_num_df_train = num_transform.fit_transform(X_train[num_col])
scaled_num_df_test = num_transform.transform(X_test[num_col])

cat_transform = preprocessing.OneHotEncoder(handle_unknown='ignore')

scaled_cat_df_train = cat_transform.fit_transform(X_train[cat_col]).toarray()
scaled_cat_df_test = cat_transform.transform(X_test[cat_col]).toarray()

In [ ]:
X_train = np.concatenate([scaled_num_df_train, scaled_cat_df_train], axis = 1)
X_test =  np.concatenate([scaled_num_df_test, scaled_cat_df_test], axis = 1)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

ну и скормим все dec tree

In [ ]:
regressor = DecisionTreeRegressor(random_state = 0) 
batch_size = int(X_train.shape[0])
idx = np.random.randint(0, high=X_train.shape[0], size=batch_size, dtype='l')  
regressor.fit(X_train[idx,:], y_train[idx])

In [ ]:
y_pred = regressor.predict(X_test)
print("MSE=",mean_squared_error(y_pred,y_test ))